    Copyright 2021 Google LLC

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        https://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.


# Dialogflow CX Bot Language Translation (Sheets Notebook)

Contains functions that are re-used in the Main Notebook

1. Functions for Reading and Writing to Sheets
2. Functions for Formatting Sheets
3. Setup/Initialize a blank Google Sheets


[Public Doc Link: Python Client for Google Sheets](https://developers.google.com/sheets/api/quickstart/python)

# Sheets

## This Notebook Env:

In [ ]:
# !python3 -V
# !python3 -m pip list | wc -l
# !python3 -m pip list | grep google

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

## Enums

In [ ]:
from enum import Enum

###########################
class SheetsName(Enum):
    CX_Lang_REF       = 1
    Training_Phrases  = 2
    Parameters        = 3
    Entities          = 4
    Flows             = 5
    Pages             = 6
    Route_Groups      = 7

###########################
class SheetsContent(Enum):
    Header     = 1
    Sub_Header = 2
    CX_Element = 3
    Def_Lang   = 4
    Content    = 5

###############################
class CX_Types(Enum):
    transition_routes       = 1
    event_handlers          = 2
    transition_route_groups = 3
    entry_fulfillment       = 4
    parameter               = 5


## Functions

In [ ]:
### According to https://developers.google.com/sheets/api/reference/limits
### Sheets API requests per user per project is 60 requests per minute (or 1 request per second)

from timeit import default_timer as timer
import time

_last_gsheets_request_time = timer()

### spacing out API calls to at most once per second since last invocation
def delay_next_gsheets_request():
    global _last_gsheets_request_time
    gap = timer() - _last_gsheets_request_time
    if __DEBUG: print(f'delay_next_gsheets_request(): time gap between last API call is {gap}s')
    if (gap < 1):
        sleep_time = round(1 - gap, 3)
        if __INFO: print(f'\tdelay_next_gsheets_request(): Sleeping {sleep_time}s as duration from last API call is {gap}s')
        time.sleep(sleep_time)
    _last_gsheets_request_time = timer()


## Functions: Sheets - Read & Write

In [ ]:
############################
def get_sheets_credentials():
    global GSheets_Creds
    if GSheets_Creds is None:
        raise Exception(f'Sheets: get_sheets_credentials(): Sheets Credentials is None, please run the Main Notebook')
    else:
        return GSheets_Creds

########################################
def batch_update_to_sheets(update_json):
    creds = get_sheets_credentials()
    service = build('sheets', 'v4', credentials=creds)
    #pprint(update_json)
    request = service.spreadsheets().batchUpdate(spreadsheetId=Google_Sheets_ID,body=update_json)
    delay_next_gsheets_request()
    response = request.execute()

#######################################
def read_sheet(sheet_range):
    creds = get_sheets_credentials()
    service = build('sheets', 'v4', credentials=creds)
    # Call the Sheets API
    sheet = service.spreadsheets()
    delay_next_gsheets_request()
    result = sheet.values().get(spreadsheetId=Google_Sheets_ID,range=sheet_range).execute()
    values = result.get('values', [])
    if not values:
        raise Exception(f'read_sheet({sheet_range}): No data found.')
    else:
        df = pd.DataFrame(values)
        df.columns = df.iloc[0]
        df.drop(df.index[0], inplace=True)
        return df

#############################
def clear_sheet(sheet_range):
    creds = get_sheets_credentials()
    service = build('sheets', 'v4', credentials=creds)
    # Call the Sheets API
    sheet = service.spreadsheets()
    delay_next_gsheets_request()
    result = sheet.values().clear(spreadsheetId=Google_Sheets_ID,range=sheet_range).execute()

##########################################################
def write_to_sheet(sheet_range, values, value_input_option, mode):
    creds = get_sheets_credentials()
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    delay_next_gsheets_request()

    if mode == 'update':
        result = sheet.values().update(spreadsheetId=Google_Sheets_ID,
                                       range=sheet_range,
                                       valueInputOption=value_input_option,
                                       body={"values":values}).execute()
        
    elif mode == 'append':
        result = sheet.values().append(spreadsheetId=Google_Sheets_ID,
                                       range=sheet_range,
                                       valueInputOption=value_input_option,
                                       insertDataOption="OVERWRITE",
                                       body={"values":values}).execute()
    else:
        raise Exception(f'write_to_sheet() mode is set to {mode} when only "update" or "append" are accepted.')
        
#####################
def colnum_string(n):
    string = ""
    while n > 0:
        n, remainder = divmod(n - 1, 26)
        string = chr(65 + remainder) + string
    return string

#####################################################
def write_result(sheet_name, column_index, row, values):
    sheet_range = f"{sheet_name}!{colnum_string(column_index)}{row}" 
    if(__DEBUG):
        print(f'write_result(): sheet_range:{sheet_range}')
    write_to_sheet(sheet_range,values,'RAW','update')


## Functions: Formatting to Sheets

In [ ]:
########################
def get_sheets_titles():
    creds = get_sheets_credentials()
    service = build('sheets', 'v4', credentials=creds)

    # The ranges to retrieve from the spreadsheet.
    ranges = []

    # True if grid data should be returned.
    # This parameter is ignored if a field mask was set in the request.
    include_grid_data = False

    request = service.spreadsheets().get(spreadsheetId=Google_Sheets_ID, 
                                         ranges=ranges, includeGridData=include_grid_data)
    delay_next_gsheets_request()
    response = request.execute()
    #pprint(response)

    sheets = response['sheets']
    #print(sheets)
    sheets_titles = []
    for sheet in sheets:
        sheets_titles.append(sheet['properties']['title'])
    return sheets_titles


##################################
def add_sheets_json(sheet_titles):
    sheets_json = []
    index = 0
    for sheet_title in sheet_titles:
        sheets_json.append({'addSheet': {'properties': {'sheetId': sheet_title.value, 'title': sheet_title.name, 'index':sheet_title.value - 1}}})
        index += 1
    return sheets_json

############################
def set_borders_json(delta):
    borders_json = []
    border = { 'style': 'SOLID',
               'width': 1,
               'color': {
                 'red': 0,
                 'blue': 0,
                 'green': 0
                }
             }
    start_row = 0
    start_col = 0
    end_row = 0
    end_col = 0
    for sheet in delta: 
        if sheet == SheetsName.CX_Lang_REF or sheet == SheetsName.Parameters:
            end_row = 1
        else:
            end_row = 3

        border_json = {'updateBorders': { 
                        'range': {
                            'sheetId': sheet.value,
                            'startRowIndex': start_row,
                            'endRowIndex': end_row,
                            'startColumnIndex': start_col
                            #'endColumnIndex': end_col
                        },
                        'top': {}, 'bottom': {},
                        'left':   {}, 'right':  {},
                        'innerHorizontal': {}, 'innerVertical':   {}
                    }
                }
        border_json['updateBorders']['top'] = border
        border_json['updateBorders']['bottom'] = border
        border_json['updateBorders']['left'] = border
        border_json['updateBorders']['right'] = border
        border_json['updateBorders']['innerHorizontal'] = border
        border_json['updateBorders']['innerVertical'] = border
        borders_json.append(border_json)
    return borders_json
    
#################################    
def set_column_width_json(delta):
    dimensions_json = []
    for sheet in delta:
        dimension_json = {'updateDimensionProperties': {
                            'properties': {'pixelSize': 150},
                            'fields': '*',
                            #'range': {'sheetId': sheet.value, 'dimension': 'COLUMNS', 'startIndex': 0, 'endIndex': end_col }
                            'range': {'sheetId': sheet.value, 'dimension': 'COLUMNS', 'startIndex': 0 }
                            }
                        }
        dimensions_json.append(dimension_json)
    return dimensions_json
        
##############################################
def cell_formatter_json(sheet, sheet_content):
    formatting = []
    range_json = {'sheetId': sheet.value,'startRowIndex': 0,'endRowIndex': {},'startColumnIndex': 0,'endColumnIndex': {} }
    cell_json = {'userEnteredFormat': {'backgroundColor':{'red':1,'green':1,'blue':1},
                                       'horizontalAlignment':'LEFT','verticalAlignment':'TOP','wrapStrategy':'WRAP',
                                       'textFormat':{'fontFamily':''}
                                    }
                }
    bg_red   = 224/255
    bg_green = 248/255
    bg_blue  = 250/255
    
    if sheet == SheetsName.CX_Lang_REF:
        del range_json['endRowIndex']
        del range_json['endColumnIndex']
        cell_json['userEnteredFormat']['wrapStrategy'] = 'OVERFLOW_CELL'
        cell_json['userEnteredFormat']['textFormat']['fontFamily'] = 'Google Sans'
        
    elif sheet == SheetsName.Training_Phrases:
        if sheet_content == SheetsContent.Header:
            range_json['endRowIndex'] = 1
            range_json['endColumnIndex'] = 2
            cell_json['userEnteredFormat']['textFormat']['fontFamily'] = 'Roboto Mono'
            cell_json['userEnteredFormat']['backgroundColor']['red'] = bg_red
            cell_json['userEnteredFormat']['backgroundColor']['green'] = bg_green
            cell_json['userEnteredFormat']['backgroundColor']['blue'] = bg_blue
        elif sheet_content == SheetsContent.Sub_Header:
            range_json['startRowIndex'] = 1
            range_json['endRowIndex'] = 3
            range_json['startColumnIndex'] = 0
            range_json['endColumnIndex'] = 2
            cell_json['userEnteredFormat']['textFormat']['fontFamily'] = 'Roboto Mono'
            cell_json['userEnteredFormat']['textFormat']['italic'] = True
            cell_json['userEnteredFormat']['horizontalAlignment'] = 'RIGHT'
            cell_json['userEnteredFormat']['backgroundColor']['red'] = bg_red
            cell_json['userEnteredFormat']['backgroundColor']['green'] = bg_green
            cell_json['userEnteredFormat']['backgroundColor']['blue'] = bg_blue
        elif sheet_content == SheetsContent.CX_Element:
            range_json['startRowIndex'] = 3
            del range_json['endRowIndex']
            range_json['endColumnIndex'] = 2
            cell_json['userEnteredFormat']['wrapStrategy'] = 'OVERFLOW_CELL'            
            cell_json['userEnteredFormat']['textFormat']['fontFamily'] = 'Roboto Mono'
            cell_json['userEnteredFormat']['backgroundColor']['red'] = bg_red
            cell_json['userEnteredFormat']['backgroundColor']['green'] = bg_green
            cell_json['userEnteredFormat']['backgroundColor']['blue'] = bg_blue
        elif sheet_content == SheetsContent.Def_Lang:
            del range_json['endRowIndex']
            range_json['startColumnIndex'] = 2
            range_json['endColumnIndex'] = 3
            cell_json['userEnteredFormat']['textFormat']['fontFamily'] = 'Google Sans'
            cell_json['userEnteredFormat']['backgroundColor']['red'] = bg_red
            cell_json['userEnteredFormat']['backgroundColor']['green'] = bg_green
            cell_json['userEnteredFormat']['backgroundColor']['blue'] = bg_blue
        elif sheet_content == SheetsContent.Content:
            del range_json['endRowIndex']
            range_json['startColumnIndex'] = 3
            del range_json['endColumnIndex']
            cell_json['userEnteredFormat']['textFormat']['fontFamily'] = 'Google Sans'
            
    elif sheet == SheetsName.Parameters:
        if sheet_content == SheetsContent.Header:
            range_json['endRowIndex'] = 1
            range_json['endColumnIndex'] = 6
            cell_json['userEnteredFormat']['textFormat']['fontFamily'] = 'Roboto Mono'
            cell_json['userEnteredFormat']['backgroundColor']['red'] = bg_red
            cell_json['userEnteredFormat']['backgroundColor']['green'] = bg_green
            cell_json['userEnteredFormat']['backgroundColor']['blue'] = bg_blue
        elif sheet_content == SheetsContent.CX_Element:
            range_json['startRowIndex'] = 1
            del range_json['endRowIndex']
            range_json['endColumnIndex'] = 6
            cell_json['userEnteredFormat']['wrapStrategy'] = 'OVERFLOW_CELL'            
            cell_json['userEnteredFormat']['textFormat']['fontFamily'] = 'Roboto Mono'
            cell_json['userEnteredFormat']['backgroundColor']['red'] = bg_red
            cell_json['userEnteredFormat']['backgroundColor']['green'] = bg_green
            cell_json['userEnteredFormat']['backgroundColor']['blue'] = bg_blue
        elif sheet_content == SheetsContent.Content:
            del range_json['endRowIndex']
            range_json['startColumnIndex'] = 7
            del range_json['endColumnIndex']
            cell_json['userEnteredFormat']['textFormat']['fontFamily'] = 'Google Sans'
        
    elif sheet == SheetsName.Pages:
        if sheet_content == SheetsContent.Header:
            range_json['endRowIndex'] = 1
            range_json['endColumnIndex'] = 6
            cell_json['userEnteredFormat']['textFormat']['fontFamily'] = 'Roboto Mono'
            cell_json['userEnteredFormat']['backgroundColor']['red'] = bg_red
            cell_json['userEnteredFormat']['backgroundColor']['green'] = bg_green
            cell_json['userEnteredFormat']['backgroundColor']['blue'] = bg_blue
        elif sheet_content == SheetsContent.Sub_Header:
            range_json['startRowIndex'] = 1
            range_json['endRowIndex'] = 3
            range_json['startColumnIndex'] = 0
            range_json['endColumnIndex'] = 6
            cell_json['userEnteredFormat']['textFormat']['fontFamily'] = 'Roboto Mono'
            cell_json['userEnteredFormat']['textFormat']['italic'] = True
            cell_json['userEnteredFormat']['horizontalAlignment'] = 'RIGHT'
            cell_json['userEnteredFormat']['backgroundColor']['red'] = bg_red
            cell_json['userEnteredFormat']['backgroundColor']['green'] = bg_green
            cell_json['userEnteredFormat']['backgroundColor']['blue'] = bg_blue
        elif sheet_content == SheetsContent.CX_Element:
            range_json['startRowIndex'] = 3
            del range_json['endRowIndex']
            range_json['endColumnIndex'] = 6
            cell_json['userEnteredFormat']['wrapStrategy'] = 'OVERFLOW_CELL'            
            cell_json['userEnteredFormat']['textFormat']['fontFamily'] = 'Roboto Mono'
            cell_json['userEnteredFormat']['backgroundColor']['red'] = bg_red
            cell_json['userEnteredFormat']['backgroundColor']['green'] = bg_green
            cell_json['userEnteredFormat']['backgroundColor']['blue'] = bg_blue
        elif sheet_content == SheetsContent.Def_Lang:
            del range_json['endRowIndex']
            range_json['startColumnIndex'] = 6
            range_json['endColumnIndex'] = 7
            cell_json['userEnteredFormat']['textFormat']['fontFamily'] = 'Google Sans'
            cell_json['userEnteredFormat']['backgroundColor']['red'] = bg_red
            cell_json['userEnteredFormat']['backgroundColor']['green'] = bg_green
            cell_json['userEnteredFormat']['backgroundColor']['blue'] = bg_blue
        elif sheet_content == SheetsContent.Content:
            del range_json['endRowIndex']
            range_json['startColumnIndex'] = 7
            del range_json['endColumnIndex']
            cell_json['userEnteredFormat']['textFormat']['fontFamily'] = 'Google Sans'

    else: #Entities, Flows & Route_Groups - same
        if sheet_content == SheetsContent.Header:
            range_json['endRowIndex'] = 1
            range_json['endColumnIndex'] = 4
            cell_json['userEnteredFormat']['textFormat']['fontFamily'] = 'Roboto Mono'
            cell_json['userEnteredFormat']['backgroundColor']['red'] = bg_red
            cell_json['userEnteredFormat']['backgroundColor']['green'] = bg_green
            cell_json['userEnteredFormat']['backgroundColor']['blue'] = bg_blue
        elif sheet_content == SheetsContent.Sub_Header:
            range_json['startRowIndex'] = 1
            range_json['endRowIndex'] = 3
            range_json['startColumnIndex'] = 0
            range_json['endColumnIndex'] = 4
            cell_json['userEnteredFormat']['textFormat']['fontFamily'] = 'Roboto Mono'
            cell_json['userEnteredFormat']['textFormat']['italic'] = True
            cell_json['userEnteredFormat']['horizontalAlignment'] = 'RIGHT'
            cell_json['userEnteredFormat']['backgroundColor']['red'] = bg_red
            cell_json['userEnteredFormat']['backgroundColor']['green'] = bg_green
            cell_json['userEnteredFormat']['backgroundColor']['blue'] = bg_blue
        elif sheet_content == SheetsContent.CX_Element:
            range_json['startRowIndex'] = 3
            del range_json['endRowIndex']
            range_json['endColumnIndex'] = 4
            cell_json['userEnteredFormat']['wrapStrategy'] = 'OVERFLOW_CELL'            
            cell_json['userEnteredFormat']['textFormat']['fontFamily'] = 'Roboto Mono'
            cell_json['userEnteredFormat']['backgroundColor']['red'] = bg_red
            cell_json['userEnteredFormat']['backgroundColor']['green'] = bg_green
            cell_json['userEnteredFormat']['backgroundColor']['blue'] = bg_blue
        elif sheet_content == SheetsContent.Def_Lang:
            del range_json['endRowIndex']
            range_json['startColumnIndex'] = 4
            range_json['endColumnIndex'] = 5
            cell_json['userEnteredFormat']['textFormat']['fontFamily'] = 'Google Sans'
            cell_json['userEnteredFormat']['backgroundColor']['red'] = bg_red
            cell_json['userEnteredFormat']['backgroundColor']['green'] = bg_green
            cell_json['userEnteredFormat']['backgroundColor']['blue'] = bg_blue
        elif sheet_content == SheetsContent.Content:
            del range_json['endRowIndex']
            range_json['startColumnIndex'] = 5
            del range_json['endColumnIndex']
            cell_json['userEnteredFormat']['textFormat']['fontFamily'] = 'Google Sans'
        
    formatting.append(range_json)
    formatting.append(cell_json)
    return formatting
    
####################################
def set_cell_formatting_json(delta):
    # Roboto Mono, Google Sans
    formats_json = []
    repeatCell_json = {'repeatCell': {'range': {},'cell': {},'fields': '*'}}
    
    for sheet in delta:
        if sheet == SheetsName.CX_Lang_REF:
            json = copy.deepcopy(repeatCell_json)
            formatting = cell_formatter_json(sheet, SheetsContent.Header)
            json['repeatCell']['range'] = formatting[0]
            json['repeatCell']['cell'] = formatting[1]
            formats_json.append(json)
            
        else: #Every other Sheets - same
            for content_type in SheetsContent:
                json = copy.deepcopy(repeatCell_json)
                formatting = cell_formatter_json(sheet, content_type)
                json['repeatCell']['range'] = formatting[0]
                json['repeatCell']['cell'] = formatting[1]
                formats_json.append(json)
                       
    return formats_json

################################
def set_view_freeze_json(delta):
    properties_jsons = []
    row_count = 3
    col_count = 0
    for sheet in delta:
        if sheet == SheetsName.CX_Lang_REF:
            row_count = 1
        elif sheet == SheetsName.Training_Phrases:
            row_count = 3
            col_count = 3
        elif sheet == SheetsName.Parameters:
            row_count = 1
            col_count = 0
        elif sheet == SheetsName.Pages:
            row_count = 3
            col_count = 7
        else:
            row_count = 3
            col_count = 5
        json = {'updateSheetProperties': 
                    {'properties':
                         {'sheetId':sheet.value,'title': sheet.name, 'index': sheet.value - 1, 
                          'gridProperties':
                              {'rowCount':1000, 'columnCount': 26,
                                'frozenRowCount': row_count,'frozenColumnCount': col_count }
                         },
                     'fields':'*'}
               }
        properties_jsons.append(json)
    return properties_jsons

#############################
def add_sheet_headers(delta):
    for d in delta:
        #print(d.name)
        if d == SheetsName.CX_Lang_REF:
            write_to_sheet(SheetsName.CX_Lang_REF.name+'!A1', 
                       [['=importhtml("https://cloud.google.com/dialogflow/cx/docs/reference/language","table",1)']], 
                       'USER_ENTERED', 'update' )
        elif d == SheetsName.Training_Phrases:
            write_to_sheet(SheetsName.Training_Phrases.name+'!A1', 
                           [['Intent Name','Intent Display Name'],[None,'CX:>'],[None,'Translate:>']], 'RAW', 'update')
        elif d == SheetsName.Parameters:
            write_to_sheet(SheetsName.Parameters.name+'!A1', 
                           [['Intent Name','Intent Display Name',
                             'Parameter ID','Parameter Entity Type',
                             'Boolean:Is_List','Boolean:Redact'],
                            [None, None, None, None, None, None],[None, None, None, None, None, None]], 'RAW', 'update')
        elif d == SheetsName.Entities:
            write_to_sheet(SheetsName.Entities.name+'!A1', 
                           [['Entity Type Name','Entity Type Display Name','Entity Type Kind','Entities Value'],
                            [None, None, None, 'CX:>'],
                            [None, None, None, 'Translate:>']
                           ], 'RAW', 'update')
        elif d == SheetsName.Flows:
            write_to_sheet(SheetsName.Flows.name+'!A1', 
                           [['Flow Name','Flow Display Name','Flow Components','Flow Component ID'],
                            [None, None, None, 'CX:>'],
                            [None, None, None, 'Translate:>']
                           ], 'RAW', 'update')
        elif d == SheetsName.Pages:
            write_to_sheet(SheetsName.Pages.name+'!A1', 
                           [['Flow Name','Page Name','Page Display Name','Page Components','Page Component ID','Page Component ID-2'],
                            [None, None, None, None, None, 'CX:>'],
                            [None, None, None, None, None, 'Translate:>']
                           ], 'RAW', 'update')
        elif d == SheetsName.Route_Groups:
            write_to_sheet(SheetsName.Route_Groups.name+'!A1', 
                           [['Flow Name','Route Group Name','Route Group Display Name','Route Group Component ID'],
                            [None, None, None, 'CX:>'],
                            [None, None, None, 'Translate:>']
                           ], 'RAW', 'update')


## Functions: CX Config to Sheets

In [ ]:
################################################################
def get_messages(fulfillment):
    messages_list = []
    messages = fulfillment.messages 
    if messages != []:
        for msg in messages:
            if 'output_audio_text' in msg:
                messages_list.append(msg.output_audio_text.ssml)
            elif 'text' in msg:
                for txt in msg.text.text:
                    messages_list.append(txt)
    return messages_list
################################################################

################################################################
def generate_sheets_values(obj):
    if __DEBUG: print(f'Type: {type(obj)}')
    sheet_values = []
    
    ### FLOW
    if type(obj) == cx_types.Flow:
        # transition_routes
        cx_component = CX_Types.transition_routes
        for tr in obj.transition_routes:
            if tr.intent != '':
                component_id = tr.intent
            elif tr.condition != '':
                component_id = tr.condition
            fulfillment = tr.trigger_fulfillment
            messages_list = get_messages(fulfillment)
            for message in messages_list:
                sheet_values.append([obj.name, obj.display_name, cx_component.name,
                                     component_id, message])

        # event_handlers
        cx_component = CX_Types.event_handlers
        for eh in obj.event_handlers:
            component_id = eh.event
            fulfillment  = eh.trigger_fulfillment
            messages_list = get_messages(fulfillment)
            for message in messages_list:
                sheet_values.append([obj.name, obj.display_name, cx_component.name,
                                     component_id, message])        
        
    ### PAGE
    elif type(obj) == cx_types.Page:
        flow_name = obj.name.split('/pages')[0]
        #entry_fulfillment
        cx_component = CX_Types.entry_fulfillment
        fulfillment = obj.entry_fulfillment        
        messages_list = get_messages(fulfillment)
        for message in messages_list:
            sheet_values.append([flow_name, obj.name, obj.display_name, cx_component.name,
                                '', '', message])
        
        #form & parameters
        cx_component = CX_Types.parameter
        parameters = obj.form.parameters
        if parameters != []:
            for param in parameters:
                # initial_prompt_fulfillment
                fulfillment = param.fill_behavior.initial_prompt_fulfillment
                messages_list = get_messages(fulfillment)
                for message in messages_list:
                    sheet_values.append([flow_name, obj.name, obj.display_name, cx_component.name,
                                        param.display_name, 'initial_prompt_fulfillment', message])
                # reprompt_event_handlers
                reprompts = param.fill_behavior.reprompt_event_handlers
                for reprompt in reprompts:
                    fulfillment = reprompt.trigger_fulfillment
                    messages_list = get_messages(fulfillment)
                    for message in messages_list:
                        sheet_values.append([flow_name, obj.name, obj.display_name, cx_component.name,
                                            param.display_name, reprompt.event, message])
        
        #transition_routes
        cx_component = CX_Types.transition_routes
        for tr in obj.transition_routes:
            if tr.intent != '':
                tr_comp_id = tr.intent
            elif tr.condition != '':
                tr_comp_id = tr.condition
            fulfillment = tr.trigger_fulfillment 
            messages_list = get_messages(fulfillment)
            for message in messages_list:
                sheet_values.append([flow_name, obj.name, obj.display_name, cx_component.name,
                                    tr_comp_id, '', message])
                    
        #event_handlers
        cx_component = CX_Types.event_handlers
        for eh in obj.event_handlers:
            fulfillment = eh.trigger_fulfillment
            messages_list = get_messages(fulfillment)
            for message in messages_list:
                sheet_values.append([flow_name, obj.name, obj.display_name, cx_component.name,
                                    eh.event, '', message])
        
    ### TRANSITION ROUTE GROUP
    elif type(obj) == cx_types.TransitionRouteGroup:
        flow_name = obj.name.split('/transitionRouteGroups')[0]
        for tr in obj.transition_routes:
            if tr.intent != '':
                tr_comp_id = tr.intent
            elif tr.condition != '':
                tr_comp_id = tr.condition
            
            fulfillment = tr.trigger_fulfillment
            messages_list = get_messages(fulfillment)
            for message in messages_list:
                sheet_values.append([flow_name, obj.name, obj.display_name,
                                    tr_comp_id, message])
        
    else:
        print('generate_sheets_values function did not match Obj to either cx_types.Flow, cx_types.Page or cx_types.TransitionRouteGroup')
    
    return sheet_values


## Initialize & Format Google Sheets

In [ ]:
#############################################
def add_all_languages_to_sheets(lang_values):
    for sheet in SheetsName:
        if sheet == SheetsName.CX_Lang_REF:
            continue
        elif sheet == SheetsName.Training_Phrases:
            write_to_sheet(sheet.name+'!C1', lang_values, 'RAW', 'update')
        elif sheet == SheetsName.Parameters:
            continue
        elif sheet == SheetsName.Pages:
            write_to_sheet(sheet.name+'!G1', lang_values, 'RAW', 'update')
        else: #Entities, Flows and Route_Groups
            write_to_sheet(sheet.name+'!E1', lang_values, 'RAW', 'update')
            
##########################
def add_langs_to_sheets():
    agent = get_agent()
    print(f'agent.default_language_code:{agent.default_language_code}')
    print(f'agent.supported_language_codes:{sorted(agent.supported_language_codes)}')
    
    lang_to_sheets = []

    lang_cx = []
    lang_cx.append(agent.default_language_code)
    sorted_supported_language_codes = sorted(agent.supported_language_codes)
    #print(sorted_supported_language_codes)
    for l in sorted_supported_language_codes:
        lang_cx.append(l)


    df = read_sheet(SheetsName.CX_Lang_REF.name)
    df = df.iloc[:,[0,1]]
    #print(df.columns)
    lang_full = []
    lang_translate = []
    for l in lang_cx:
        #lang_df = df[df['Tag *'].str.lower()==l]
        lang_full.append(df[df['Tag *'].str.lower()==l].iloc[0,0])
        index = l.find('-')
        if index == -1:
            lang_translate.append(l)
        elif l == 'zh-cn':
            lang_translate.append('zh-CN')
        elif l.startswith('zh') and l.endswith('tw') or l.endswith('hk'):
            lang_translate.append('zh-TW')
        else:
            lang_translate.append(l[:index])

    lang_full[0] = f'Default:[{lang_full[0]}]'
    lang_to_sheets.append(lang_full)
    lang_to_sheets.append(lang_cx)
    lang_to_sheets.append(lang_translate)
    #print(lang_to_sheets)
    add_all_languages_to_sheets(lang_to_sheets)

In [ ]:
#########################
def init_format_sheets():
    start_time = timer()
    print("START: init_format_sheets()")
    # Add Sheets to Sheet
    # and other formatting via a single BatchUpdate
    # Enum class SheetsName = ['CX_Lang_REF', 'Training_Phrases', 'Parameters', 'Entities', 'Flows', 'Pages', 'Route_Groups']
    batch_update_req = {'requests':[]}

    sheets_titles = get_sheets_titles()
    delta = [item for item in SheetsName if item.name not in sheets_titles]
    if len(delta) > 0:
        print(f"Adding Sheets:{delta}")
        batch_update_req['requests'].append(add_sheets_json(delta))
        batch_update_req['requests'].append(set_column_width_json(delta))
        batch_update_req['requests'].append(set_cell_formatting_json(delta))
        batch_update_req['requests'].append(set_borders_json(delta))
        batch_update_req['requests'].append(set_view_freeze_json(delta))
        #pprint(batch_update_req)
        batch_update_to_sheets(batch_update_req)
        add_sheet_headers(delta)
        
        ## add_langs_to_sheets() - now folded into init_format_sheets function due to conundrum below if it is run separately
        ## Queries Agent to get default and supported languages
        ## Adds to the Sheets columns for languages
        ## If Sheet is already initialized, it still adds the languages
        ## Could be a problem if the Agent supported languages changed and the columns gets re-written and the translation (if done previously) is out of sync with column
        ## also figures out the lang tag for Translate (might be slightly different from CX language tag)
        add_langs_to_sheets()
        print()
    else:
        sheets_names = []
        for s in SheetsName:
            sheets_names.append(s.name)
        print(f'init_format_sheets():  NO actions taken as ALL required Sheets were found:\n{sheets_names}')
    
    print(f"COMPLETED: init_format_sheets() in {timer() - start_time}s")

# END

In [ ]:
print('Sheets Notebook: RAN successfully to desired point')